## Stage1:生成模型



#### 导入包

In [1]:
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data, io
from skimage.color import label2rgb
import skimage
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
%matplotlib inline
import glob as gb

In [2]:
def LBPhist(image):
    # settings for LBP
    radius = 3
    n_points = 8 * radius
    # 转换为灰度图显示
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 处理
    lbp = local_binary_pattern(image, n_points, radius)
    bins = np.arange(257)
    hist,bins = np.histogram(lbp,bins)
    return hist

### 测试LBP

#### train_smoke

In [3]:
img_path_train_smoke = gb.glob("F:\\smokeproject\\sets\\set1\\smoke\\*.jpg") 
num_length_train_smoke = len(img_path_train_smoke)
hist_empty = np.zeros([1,256])
y_train_smoke = np.ones([num_length_train_smoke,1])

for path in img_path_train_smoke:
    img  = cv2.imread(path) 
    hist = LBPhist(img)
    hist_empty = np.vstack((hist_empty, hist))
    
train_smoke = hist_empty[1:,:]

In [4]:
##test
hist_empty[1:,:]

array([[119.,   5.,   1., ...,   0.,   4.,  12.],
       [119.,   8.,   0., ...,   0.,   6.,  16.],
       [ 98.,   4.,   0., ...,   0.,   0.,   6.],
       ...,
       [ 81.,   7.,   1., ...,   0.,   4.,   3.],
       [ 79.,   2.,   1., ...,   0.,   5.,   8.],
       [104.,   1.,   0., ...,   0.,   4.,   6.]])

#### train_no_smoke

In [5]:
img_path_train_no_smoke = gb.glob("F:\\smokeproject\\sets\\set1\\non\\*.jpg") 
num_length_train_no_smoke = len(img_path_train_no_smoke)
hist_empty = np.zeros([1,256])
y_train_no_smoke = np.zeros([num_length_train_no_smoke,1])

for path in img_path_train_no_smoke:
    img  = cv2.imread(path) 
    hist = LBPhist(img)
    hist_empty = np.vstack((hist_empty, hist))
    
train_no_smoke = hist_empty[1:,:]

train_no_smoke

array([[138.,   9.,   1., ...,   0.,   4.,   7.],
       [175.,  17.,   3., ...,   0.,   2.,   4.],
       [134.,   2.,   0., ...,   0.,   1.,   0.],
       ...,
       [188.,  11.,   5., ...,   0.,   0.,   0.],
       [155.,   6.,   0., ...,   0.,   0.,   2.],
       [175.,  10.,   4., ...,   0.,   1.,   0.]])

#### test_smoke

In [6]:
img_path_test_smoke = gb.glob("F:\\smokeproject\\sets\\set2\\smoke\\*.jpg") 
num_length_test_smoke = len(img_path_test_smoke)
hist_empty = np.zeros([1,256])
y_test_smoke = np.ones([num_length_test_smoke,1])

for path in img_path_test_smoke:
    img  = cv2.imread(path) 
    hist = LBPhist(img)
    hist_empty = np.vstack((hist_empty, hist))
    
test_smoke = hist_empty[1:,:]

#### test_no_smoke

In [7]:
img_path_test_no_smoke = gb.glob("F:\\smokeproject\\sets\\set2\\non\\*.jpg") 
num_length_test_no_smoke = len(img_path_test_no_smoke)
hist_empty = np.zeros([1,256])
y_test_no_smoke = np.zeros([num_length_test_no_smoke,1])

for path in img_path_test_no_smoke:
    img  = cv2.imread(path) 
    hist = LBPhist(img)
    hist_empty = np.vstack((hist_empty, hist))
    
test_no_smoke = hist_empty[1:,:]

test_no_smoke

array([[217.,  10.,   2., ...,   0.,   0.,   2.],
       [113.,  12.,   5., ...,   0.,   3.,   5.],
       [153.,   8.,   2., ...,   0.,   1.,   2.],
       ...,
       [101.,  13.,   2., ...,   0.,   3.,   5.],
       [ 92.,  18.,   2., ...,   0.,   1.,  11.],
       [113.,  16.,   3., ...,   0.,   2.,   3.]])

#### 连接矩阵

In [8]:
#np.concatenate((a,b),axis=0)

In [9]:
#连接矩阵并转换格式
y_train = np.vstack((y_train_smoke, y_train_no_smoke))
X_train = np.vstack((train_smoke,train_no_smoke))

y_train = y_train.astype(np.int64)

In [10]:
y_test = np.vstack((y_test_smoke, y_test_no_smoke))
X_test = np.vstack((test_smoke,test_no_smoke))

y_test = y_test.astype(np.int64)

In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1383, 256)
(1383, 1)
(1505, 256)
(1505, 1)


#### 打乱排序

In [10]:
#all_xys = np.hstack((X_train,y_train))

In [19]:
indices = np.random.permutation(X_train.shape[0]) # shape[0]表示第0轴的长度，通常是训练数据的数量
rand_data_x = X_train[indices]
rand_data_y = y_train[indices] # data_y就是标记（label）

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, Y_train, Y_test = train_test_split(rand_data_x, rand_data_y, test_size = 0.25, random_state = 0)


#### SVM-RBF 测试

In [12]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

E:\Anaconda\envs\ml2\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anaconda\envs\ml2\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [13]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix 混淆矩阵
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [14]:
cm

array([[766,  51],
       [ 67, 621]], dtype=int64)